In [142]:
import pandas as pd
import numpy as np
train_alldata = pd.read_csv("./5gram-edits-train-sents-clean.csv")
dev_alldata = pd.read_csv("./5gram-edits-dev-sents-clean.csv")
test_alldata = pd.read_csv("./5gram-edits-test-sents-clean.csv")

def data_stripper(arr, included_cols=['7','9']):
    new_data = arr[included_cols]
    new_data.columns = ['Bias Inducing Word', 'Sentence Pre-Edit']
    return new_data

train,dev,test = data_stripper(train_alldata),data_stripper(dev_alldata),data_stripper(test_alldata)
print(train[5:10])

  Bias Inducing Word                                  Sentence Pre-Edit
5         Protestant  The Primate of All Ireland is the Archbishop o...
6             denied  The Church of Ireland claims Apostolic success...
7          followed.    Relatively few parish clergy or laity followed.
8      denomination.  Today the Church of Ireland is, after the Roma...
9        institution  The Church of Ireland came into existence as a...


In [144]:
bias_lexicon_file = open('./bias-lexicon/bias-lexicon.txt','r')
implicatives_file = open('./bias_related_lexicons/implicatives_karttunen1971.txt','r')
assertives_file = open('./bias_related_lexicons/assertives_hooper1975.txt','r')
factives_file = open('./bias_related_lexicons/factives_hooper1975.txt','r')
hedges_file = open('./bias_related_lexicons/hedges_hyland2005.txt','r')
other_file = open('./bias_related_lexicons/other_lexicons.txt','r')
report_verbs_file = open('./bias_related_lexicons/report_verbs.txt','r')
entailments_file = open('./entailments/reverb_global_clsf_all_tncf_lambda_0.1.txt','r')
bias_lexicon = bias_lexicon_file.read().strip().split('\n')
assertives = assertives_file.read().strip().split('\n')[7:]
factives = factives_file.read().strip().split('\n')[7:]
hedges = hedges_file.read().strip().split('\n')[7:]
other_lexicon = other_file.read().strip().split('\n')
report_verbs = report_verbs_file.read().strip().split('\n')[9:]
entailments_prestrip = entailments_file.read().strip().split('\n')

def entailment_sorter(arr, length_entailing_predicate = 1, orderXY=True):
    '''
    Takes entailment dataset and distills it into usable information. Use params to get
    the output you want. X 'word' Y = True means first argument is X, second is Y. False
    means first argument is Y and second is X. 
    If orderXY = True it includes the last 2 headers:
    entailing predicate, entailed predicate, X.Y=T/F entailing pred., X.Y=T/F entailed pred.
    '''
    if orderXY:
        data = []
        for e in arr:
            x, y = e.split('\t')
            if len(x.split()) <= length_entailing_predicate:
                x_arg, y_arg = True, True
                if '@R@' in x: x_arg = False
                if '@R@' in y: y_arg = False
                data.append([x.replace('@R@',''), y.replace('@R@',''), x_arg, y_arg])
        df = pd.DataFrame(data, columns=['Entailing Predicate','Entailed Predicate',
                                         'X.Y=T/F Entailing Pred.','X.Y=T/F Entailed Pred.'])
        return df
    else:
        data = []
        for e in arr:
            x, y = e.split('\t')
            if len(x.split()) <= length_entailing_predicate:
                data.append([x.replace('@R@',''), y.replace('@R@','')])
        df = pd.DataFrame(data, columns=['Entailing Predicate','Entailed Predicate'])
        return df

entailments = entailment_sorter(entailments_prestrip, length_entailing_predicate = 1, 
                                orderXY=True)
entailing_pred = list(entailments['Entailing Predicate'])

print(len(bias_lexicon), len(assertives), len(factives), len(hedges), 
      len(other_lexicon), len(report_verbs), len(entailing_pred))

['sweep', 'revolutionize', 'attribute', 'presume', 'experience', 'undermine', 'recall', 'charge', 'customize', 'contract']
654 66 27 100 11 181 18070


In [157]:
def dummy_vars(arr,tag_index=10):
    POS_tags = np.array(['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS',
                'NNP','NNPS','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','TO','UH',
               'VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB'])
    
    word_tag_tuples = arr[tag_index]
    tag_vector = np.zeros(len(POS_tags))
    for i in range(len(word_tag_tuples)):
        word = word_tag_tuples[i][0]
        tag = word_tag_tuples[i][1]
        tag_vector_index = np.where(tag == POS_tags)
        tag_vector[tag_vector_index] += 1
    return tag_vector

In [159]:
b = [2,[('Man','NNP'),('Joe','VBN')]]
dummy_vars(b,tag_index=1)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0.])